<a href="https://colab.research.google.com/github/mugurm/colab/blob/main/mugurm_stock_explorer_colab_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Explorer (Automatic Trailing Stop edition)
by mugurm

In [1]:
# Import the necessary libraries
import yfinance as yf
import holoviews as hv
import pandas as pd
import numpy as np
from holoviews import opts
from ipywidgets import widgets, Layout
from datetime import datetime, timedelta
from IPython.display import display, clear_output

# Use bokeh backend
hv.extension('bokeh')    

def download_data(symbol, start, end):
    """Fetch stock data using yfinance library."""
    try:
        df = yf.download(symbol, start=start, end=end)
        return df
    except Exception as e:
        print(f"Failed to fetch data for {symbol}. Error: {str(e)}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

def calculate_trailing_stops(df, trailing_stop_pct):
    """Calculate the trailing stop levels based on the highest price reached so far."""
    highest_price = df['High'].expanding().max()
    trailing_sell_level = highest_price * (1 - trailing_stop_pct / 100)
    df['Trailing_Sell_Level'] = trailing_sell_level
    return df
    

def find_buy_and_sell_signals(df, trailing_buy_pct, trailing_sell_pct):
    """Find the buy and sell signals."""
    buy_signal = (df.Low <= df.Trailing_Sell_Level)
    buy_date, buy_price, sell_date, sell_price = None, None, None, None

    if buy_signal.any():
        buy_date = buy_signal.idxmax()  # date of first True
        buy_price = df.loc[buy_date, 'Low']
        df_after_buy = df.loc[buy_date:].copy()
        df_after_buy['Trailing_Buy_Level'] = df_after_buy['High'].expanding().min() * (1 + trailing_buy_pct / 100)

        # Get the next date after buy_date, and terminate the "Trailing_Sell_Level" curve from that date onwards
        next_day = df.loc[buy_date:].index[1] if (len(df.loc[buy_date:].index) > 1) else buy_date
        df.loc[next_day:, 'Trailing_Sell_Level'] = np.nan
        
        # Terminate the "Trailing_Sell_Level" curve in the original df and df_after_buy from buy_date onwards
        df_after_buy['Trailing_Sell_Level'] = np.nan


        sell_signal = (df_after_buy.High >= df_after_buy.Trailing_Buy_Level)

        if sell_signal.any():
            sell_date = sell_signal.idxmax()  # date of first True
            sell_price = df_after_buy.loc[sell_date, 'High']

            # Get the next date after sell_date, and terminate the "Trailing_Buy_Level" curve from that date onwards
            next_day = df_after_buy.loc[sell_date:].index[1] if (len(df_after_buy.loc[sell_date:].index) > 1) else sell_date
            df_after_buy.loc[next_day:, 'Trailing_Buy_Level'] = np.nan

        df = df_after_buy.combine_first(df)  # give priority to df_after_buy and merge the new trailing levels into the original dataframe
    return buy_date, buy_price, sell_date, sell_price, df


def create_plot(df, buy_date, buy_price, sell_date, sell_price):
    """Create and display the plot."""
    price_curve = hv.Curve(df, ('Date', 'Date'), ('High', 'High')).opts(color='black') * \
                  hv.Curve(df, ('Date', 'Date'), ('Low', 'Low')).opts(color='black')
    sell_curve = hv.Curve(df, ('Date', 'Date'), ('Trailing_Sell_Level', 'Trailing Sell Level')).opts(color='red')

    # Only plot Trailing_Buy_Level if it exists
    if 'Trailing_Buy_Level' in df.columns:
        buy_curve = hv.Curve(df, ('Date', 'Date'), ('Trailing_Buy_Level', 'Trailing Buy Level')).opts(color='green')
    else:
        buy_curve = hv.Curve([])  # Empty placeholder to not disrupt the plot

    buy_point = hv.Scatter([(buy_date, buy_price)] if buy_date else []).opts(color='red', size=10)
    sell_point = hv.Scatter([(sell_date, sell_price)] if sell_date else []).opts(color='green', size=10)

    display((price_curve * sell_curve * buy_curve * buy_point * sell_point).opts(width=600))

def plot_stock(symbol, days, trailing_stop_pct_buy, trailing_stop_pct_sell, days_offset):
    """Fetch and plot the stock data."""
    end = datetime.now()
    start = end - timedelta(days=days)

    # Fetch data
    df = download_data(symbol, start, end)
    if df.empty:
        print(f"No data available for {symbol} in the given date range.")
        return

    # Calculate trailing stops
    df = calculate_trailing_stops(df, trailing_stop_pct_sell)

    # Find buy and sell signals
    buy_date, buy_price, sell_date, sell_price, df = find_buy_and_sell_signals(df, trailing_stop_pct_buy, trailing_stop_pct_sell)

    # Create and display the plot
    create_plot(df, buy_date, buy_price, sell_date, sell_price)

# Create input widgets
symbol_input = widgets.Text(value='GOOG', description='Stock Symbol:')
days_input = widgets.IntSlider(layout=Layout(width='500px'), value=180, min=20, max=360, step=10, description='Days:')
days_offset = widgets.IntSlider(layout=Layout(width='500px'), value=350, min=10, max=350, step=1, description='Start day:')
trailing_stop_pct_buy_input = widgets.FloatSlider(layout=Layout(width='500px'), value=1.0, min=0.1, max=25.0, step=0.1, description='Buy Trailing Stop (%):')
trailing_stop_pct_sell_input = widgets.FloatSlider(layout=Layout(width='500px'), value=1.0, min=0.1, max=25.0, step=0.1, description='Sell Trailing Stop (%):')

# Define the event handler
def on_input_change(change):
    clear_output(wait=True)
    display(symbol_input, days_input, trailing_stop_pct_buy_input, trailing_stop_pct_sell_input, days_offset)
    plot_stock(symbol_input.value, days_input.value, trailing_stop_pct_buy_input.value, trailing_stop_pct_sell_input.value, days_offset.value)

# Attach the event handler to the 'on_submit' event and 'value' change event
symbol_input.on_submit(on_input_change)
days_input.observe(on_input_change, names='value')
days_offset.observe(on_input_change, names='value')
trailing_stop_pct_buy_input.observe(on_input_change, names='value')
trailing_stop_pct_sell_input.observe(on_input_change, names='value')

# Display the widgets and initial plot
display(symbol_input, days_input, trailing_stop_pct_buy_input, trailing_stop_pct_sell_input, days_offset)
plot_stock(symbol_input.value, days_input.value, trailing_stop_pct_buy_input.value, trailing_stop_pct_sell_input.value, days_offset.value)

Text(value='QQQ', description='Stock Symbol:')

IntSlider(value=150, description='Days:', layout=Layout(width='500px'), max=360, min=20, step=10)

FloatSlider(value=0.1, description='Buy Trailing Stop (%):', layout=Layout(width='500px'), max=25.0, min=0.1)

FloatSlider(value=9.8, description='Sell Trailing Stop (%):', layout=Layout(width='500px'), max=25.0, min=0.1)

IntSlider(value=350, description='Start day:', layout=Layout(width='500px'), max=350, min=10)

[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I    :Curve   [Date]   (High)
   .Curve.II   :Curve   [Date]   (Low)
   .Curve.III  :Curve   [Date]   (Trailing_Sell_Level)
   .Curve.IV   :Curve   [x]   (y)
   .Scatter.I  :Scatter   [x]   (y)
   .Scatter.II :Scatter   [x]   (y)

You are a highly skilled, staff Google engineer and expert in python and data visualization. You are also very familiar with stock market trading and technical analysis and have programmed many automated trading systems and simulations. You know all of the concepts on Investopedia. You will help me integrate the days_offset input to shift the start of the trailing stop calculation to the following Google Colab notebook: